In [2]:
import sys;
from esinet.evaluate import eval_mse
# sys.path.insert(0, '../')
from esinet import Simulation, Net, util, evaluate
from forward import create_forward_model, get_info
from matplotlib import pyplot as plt
from esinet.util import calculate_source
import numpy as np
import mne

In [3]:
# Create generic Forward Model 设置频率
info = get_info(sfreq=256)
# 生成前向模型，ico2表示324个源，ico3表示1284个源，oct5表示2052个源,其间距为10mm ico4表示5124个源,
fwd = create_forward_model(info=info, sampling='ico3')
leadfield, pos = util.unpack_fwd(fwd)[1:3]
print(fwd)
fwd.save('fwd.fif', overwrite=True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    2.5s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    2.5s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    2.5s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    2.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


<Forward | MEG channels: 0 | EEG channels: 32 | Source space: Surface with 1284 vertices | Source orientation: Fixed>
    Write a source space...
    [done]


[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    1.4s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    1.5s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    1.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.5s finished
<ipython-input-3-273f23927ef3>:7: RuntimeWarning: This filename (fwd.fif) does not conform to MNE naming conventions. All forward files should end with -fwd.fif, -fwd.fif.gz, _fwd.fif, _fwd.fif.gz, -fwd.h5 or _fwd.h5
  fwd.save('fwd.fif', overwrite=True)


    Write a source space...
    [done]
    2 source spaces written


<ipython-input-3-273f23927ef3>:7: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  fwd.save('fwd.fif', overwrite=True)


In [3]:
settings = dict(duration_of_trial=0.1)
sim = Simulation(fwd, info, settings=settings, verbose=True)
sim.simulate(n_samples=20000)

Simulate Source
Simulating data based on sparse patches.


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [05:56<00:00, 56.17it/s]


Converting Source Data to mne.SourceEstimate object


100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:03<00:00, 5263.18it/s]


source data shape:  (1284, 26) (1284, 26)

Project sources to EEG...

Create EEG trials with noise...


100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [01:52<00:00, 177.55it/s]



Convert EEG matrices to a single instance of mne.Epochs...


In [4]:
# 提取源信号
sources = sim.source_data
sources = [source.data for source in sources]
print(type(sources))
print(type(sources[0]))

<class 'list'>
<class 'numpy.ndarray'>


In [5]:
y=np.array(sources)
print(type(y))
print(y.shape)

<class 'numpy.ndarray'>
(20000, 1284, 26)


In [6]:
x=np.array(sim.eeg_data)
print(type(x))
print(x.shape)
x=x.squeeze()
print(x.shape)

<class 'numpy.ndarray'>
(20000, 1, 32, 26)
(20000, 32, 26)


In [7]:
np.save('x.npy', x)

In [8]:
np.save('y.npy', y)

In [9]:
np.save('pos.npy', pos)

In [10]:
import pickle

# 指定要保存到的文件名
file_name = 'sim.pkl'

# 打开文件，使用二进制写模式
with open(file_name, 'wb') as f:
    # 使用pickle.dump()函数将对象保存到文件中
    pickle.dump(sim, f)